# MTMount accelerometers
Comparing MTMount accelerometers to accelerations from encoders\
Craig Lage - 22-Mar-23 \
Edited by Elana Urbach \
Edited by Craig Lage

In [ ]:
import sys, time, os, asyncio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client.efd_helper import EfdClient, merge_packed_time_series

##  Functions for calculating the accelerations

In [ ]:
baseFields = ['accelerationX', 'accelerationY', 'accelerationZ']
sensorNames = ['SST top end ring +x -y', 'SST top end ring -x -y', 'SST spider spindle', 'SST M2 surrogate'] 

def Locations(sensorName):
    # This carries the sensor location information
    if sensorName == 'SST top end ring +x -y':
        X = 3.876; Y = -3.696; Z = 4.65
    if sensorName == 'SST top end ring -x -y':
        X = -3.786; Y = -3.786; Z = 4.653
    if sensorName == 'SST spider spindle':
        X = -1.275; Y = 0.602; Z = 5.383
    if sensorName == 'SST M2 surrogate':
        X = -1.882; Y = 0.0; Z = 4.342
    r_theta = np.sqrt(Z**2 + Y**2)
    r_phi = np.sqrt(Z**2 + Y**2 + X**2)
    return [X, Y, Z, r_theta, r_phi]
    
def EncoderAccels(az, el, start_slew, inPos):
    # This calculates the encoder accelerations
    smoothingFactor = 0.2 # In spline creation
    az = az[(az['timestamp'] > (start_slew - 2.0)) & (az['timestamp'] < (inPos + 1.0))]
    el = el[(el['timestamp'] > (start_slew - 2.0)) & (el['timestamp'] < (inPos + 1.0))]
    azVs = az['actualVelocity'].values
    azXs = az['timestamp'].values - az['timestamp'].values[0]  
    elVs = el['actualVelocity'].values
    elXs = el['timestamp'].values - el['timestamp'].values[0]
    timesAz = np.linspace(azXs[0], azXs[-1], len(azXs)*100)
    timesEl = np.linspace(elXs[0], elXs[-1], len(elXs)*100)
    kernel_size = 200 # In convolution
    kernel = np.ones(kernel_size) / kernel_size

    azVelSpline1 = UnivariateSpline(azXs, azVs, s=0) 
    # Now smooth the derivative before differentiating
    smoothedAzVel = np.convolve(azVelSpline1(timesAz), kernel, mode='same')
    azVelSpline = UnivariateSpline(timesAz, smoothedAzVel, s=smoothingFactor)
    azAccSpline1 = azVelSpline.derivative(n=1)
    smoothedAzAcc = np.convolve(azAccSpline1(timesAz), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    azAccSpline = UnivariateSpline(timesAz, smoothedAzAcc, s=smoothingFactor)
    accelsAz = azAccSpline(timesAz)
    elVelSpline1 = UnivariateSpline(elXs, elVs, s=0)
    # Now smooth the derivative before differentiating 
    smoothedElVel = np.convolve(elVelSpline1(timesEl), kernel, mode='same')
    elVelSpline = UnivariateSpline(timesEl, smoothedElVel, s=smoothingFactor)
    elAccSpline1 = elVelSpline.derivative(n=1)
    smoothedElAcc = np.convolve(elAccSpline1(timesEl), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    elAccSpline = UnivariateSpline(timesEl, smoothedElAcc, s=smoothingFactor)
    accelsEl = elAccSpline(timesEl)
    return [timesAz, accelsAz, timesEl, accelsEl]

def AccelerometerAccels(packed_dataframe, el, az, sensorName, start_slew, inPos):
    # This rotates the accelerometer accelerations into AzEl space
    g = 9.5
    rolling_average = 100 # Smooths the noisy data   
    sub_df = packed_dataframe.loc[packed_dataframe.sensorName==sensorName] 
    sub_df = sub_df[(sub_df['timestamp'] > (start_slew - 2.0)) & (sub_df['timestamp'] < (inPos + 1.0))]
    el = el[(el['timestamp'] > (start_slew - 2.0)) & (el['timestamp'] < (inPos + 1.0))]
    elevations = np.array(el.actualPosition[np.argmin(np.abs(np.subtract(np.array(el.timestamp), \
                                np.expand_dims(np.array(sub_df.timestamp), 0).T)), axis = 1)])
    unpacked_elevations = np.repeat(elevations, 200) * np.pi/180.0 # In radians
    
    az = az[(az['timestamp'] > (start_slew - 2.0)) & (az['timestamp'] < (inPos + 1.0))]
    azimuth = np.array(az.actualPosition[np.argmin(np.abs(np.subtract(np.array(az.timestamp), \
                                np.expand_dims(np.array(sub_df.timestamp), 0).T)), axis = 1)])
    unpacked_azimuth = np.repeat(azimuth, 200)
    
    [X, Y, Z, r_theta, r_phi] = Locations(sensorName) # Get the sensor coordinates
    r_phi_axis = np.sqrt((Z * np.cos(unpacked_elevations) \
                          - Y * np.sin(unpacked_elevations))**2 \
                          + X**2)
    for baseField in baseFields:
        df = merge_packed_time_series(sub_df, baseField, stride=1,
                             ref_timestamp_col="timestamp", fmt='unix_tai',
                             scale='tai')
        if baseField == 'accelerationX':
            az_el_accel_df = pd.DataFrame(data = {'AccelerationElevation': np.zeros(len(df.index)), \
                                                  'AccelerationAzimuth': np.zeros(len(df.index)), \
                                                  'times': df['times'] - df['times'].values[0]})
            az_el_accel_df['Elevation'] = unpacked_elevations * 180/np.pi
            az_el_accel_df['Azimuth'] = unpacked_azimuth 
            az_el_accel_df['AccelerationAzimuth'] -= (np.array(df['accelerationX']) * (Z * np.cos(unpacked_elevations) \
                                                  - Y * np.sin(unpacked_elevations)) / (r_phi_axis**2)) * 180.0/np.pi #-
        if baseField == 'accelerationY':
            df['accelerationY'] -=  g * np.cos(unpacked_elevations)
            az_el_accel_df['AccelerationElevation'] += (np.array(df['accelerationY']) * Z / (r_theta**2)) * 180.0/np.pi
            az_el_accel_df['AccelerationAzimuth'] -= (np.array(df['accelerationY'])) * np.sin(unpacked_elevations) \
                                                        * (X / (r_phi_axis**2)) * 180.0/np.pi #-
        if baseField == 'accelerationZ':
            df['accelerationZ'] -=  g * np.sin(unpacked_elevations)
            az_el_accel_df['AccelerationElevation'] -= (np.array(df['accelerationZ']) * Y / (r_theta**2)) * 180.0/np.pi 
            az_el_accel_df['AccelerationAzimuth'] += (np.array(df['accelerationZ']) * np.cos(unpacked_elevations) \
                                                           * X / (r_phi_axis**2)) * 180.0/np.pi #+

    # Now subtract off the mean.  This shouldn't be necessary if we have successfully removed g
    az_el_accel_df['AccelerationAzimuth'] -= az_el_accel_df['AccelerationAzimuth'].mean()
    az_el_accel_df['AccelerationElevation'] -= az_el_accel_df['AccelerationElevation'].mean()
    # Now do a rolling average to smooth the data
    az_el_accel_df['AccelerationAzimuth'] = az_el_accel_df['AccelerationAzimuth'].rolling(rolling_average, center=True).sum() / rolling_average 
    az_el_accel_df['AccelerationElevation'] = az_el_accel_df['AccelerationElevation'].rolling(rolling_average, center=True).sum() / rolling_average
    # az_el_accel_df['AccelerationAzimuth'] = az_el_accel_df['AccelerationAzimuth']
    # az_el_accel_df['AccelerationElevation'] = az_el_accel_df['AccelerationElevation']
    return az_el_accel_df

##  Get all of the data for the selected times

In [ ]:
# Get EFD client
client = EfdClient('usdf_efd')

In [ ]:
# Times to look at the data
start = Time("2023-03-25T01:26:29", scale='utc')
end = Time("2023-03-25T01:46:02", scale='utc')

# start = Time("2023-03-23T08:00:00", scale='utc') # large azimuth slews
# end = Time("2023-03-23T09:25:30", scale='utc') # large azimuth slews

In [ ]:
packed_dataframe = await client.select_time_series("lsst.sal.ESS.accelerometer", ["*"], start, end)

az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                        ['actualPosition', 'actualVelocity', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                        ['actualPosition', 'actualVelocity', 'timestamp'],  start, end)

az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)   

azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
azPos = azPos[azPos['inPosition']] # Select only the True values
elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
elPos = elPos[elPos['inPosition']] # Select only the True values

slew_starts = await client.select_time_series('lsst.sal.MTPtg.command_raDecTarget', \
                                            ['ra', 'declination', 'private_sndStamp'],  start, end)


## Now find the slew times and pair them up

In [ ]:
# Find all of the time stamps
# Start with start_slew times

azs = az_track.values[:,0]
els = el_track.values[:,0]
times = az_track.values[:,1]
start_slew_times_1 = []
slew_dist_1 = []
slew_times_1 = []
for i in range(1,len(az_track)):
    az_shift = abs(azs[i] - azs[i-1])
    el_shift = abs(els[i] - els[i-1])
    if (az_shift > 0.1) or (el_shift > 0.1):
        start_slew_times_1.append(times[i])
        az_shift_mod = az_shift * np.cos(els[i]*np.pi/180.0)
        shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
        slew_dist_1.append(shift)
print(len(start_slew_times_1))

# Now in position timestamps

inPos_1 = []
azPosValues = azPos.values[:,1]
elPosValues = elPos.values[:,1]

for i in range(len(azPos)):
    if azPosValues[i] > elPosValues[i]:
        inPos_1.append(azPosValues[i])
    else:
        inPos_1.append(elPosValues[i])

print(len(inPos_1))

# Now pair them up

pairMin = 1.0
pairMax = 6.0

start_slew_times = []
slew_dist = []
slew_times = []
inPos = []

for i in range(len(start_slew_times_1)):
    for j in range(len(inPos_1)):
        deltaT = inPos_1[j] - start_slew_times_1[i] 
        if deltaT > pairMin and deltaT < pairMax:
            inPos.append(inPos_1[j])
            start_slew_times.append(start_slew_times_1[i])
            slew_times.append(deltaT)
            slew_dist.append(slew_dist_1[i])
            
time_deltas = np.zeros(len(start_slew_times))
slew_dist = np.array(slew_dist)
slew_times = np.array(slew_times)

for i, start_slew_time in enumerate(start_slew_times):
        k = np.argmin(np.abs((Time(start_slew_time, format='unix_tai', scale='utc') - \
                              Time(slew_starts.private_sndStamp, format='unix_tai', scale='utc')).to_value('s')))
        time_delta = (Time(start_slew_time, format='unix_tai', scale='utc') - \
                      Time(slew_starts.private_sndStamp[k], format='unix_tai', scale='utc'))
        time_deltas[i] = time_delta.to_value('s')
        start_slew_times[i] = slew_starts.private_sndStamp[k]
        
slew_times = slew_times + time_deltas
        
print(len(inPos), len(start_slew_times), len(slew_times), len(slew_dist))

## Now run the accelerations and plot the comparisons

In [ ]:
for index in range(len(inPos)):
    # plt.figure(figsize = (16,10))
    plt.figure(figsize = (16,4))
    plt.subplots_adjust(hspace=0.5)
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    plt.suptitle(f"EFD vs Accelerometer accelerations - {ip_time}")
    for sensorName in sensorNames:
        az_el_accel_df = AccelerometerAccels(packed_dataframe, el, az, sensorName, start_slew_times[index], inPos[index])
        plt.subplot(1,3,1)
        plt.title('Elevation')
        plt.plot(az_el_accel_df['times'], az_el_accel_df['AccelerationElevation'], label= str(sensorName))
        plt.subplot(1,3,2)
        plt.title('Azimuth')
        plt.plot(az_el_accel_df['times'], az_el_accel_df['AccelerationAzimuth'], label= str(sensorName))
        '''
        plt.subplot(2,3,4)
        plt.title('Elevation')
        plt.plot(az_el_accel_df['times'], az_el_accel_df['Elevation'], label= str(sensorName))
        plt.subplot(2,3,5)
        plt.title('Azimuth')
        plt.plot(az_el_accel_df['times'], az_el_accel_df['Azimuth'], label= str(sensorName))
        '''
        
    [timesAz, accelsAz, timesEl, accelsEl] = EncoderAccels(az, el, start_slew_times[index], inPos[index])
    plt.subplot(1,3,1)
    plt.plot(timesEl, accelsEl, label = 'MT encoder', ls='--', lw=2, color='k')
    plt.plot([2.0,2.0], [np.min(accelsEl), np.max(accelsEl)], ls=':', color='r', label="Slew start")
    plt.plot([timesEl[-1]-1.0, timesEl[-1]-1.0], [np.min(accelsEl), np.max(accelsEl)], ls=':', color='g', label="InPos")
    plt.xlabel("Time(sec)")
    plt.ylabel("Accel(deg/s^2)")
    plt.subplot(1,3,2)
    plt.plot(timesAz, accelsAz, label = 'MT encoder', ls='--', lw=2, color='k')
    plt.plot([2.0,2.0], [np.min(accelsAz), np.max(accelsAz)], ls=':', color='r', label="Slew start")
    plt.plot([timesAz[-1]-1.0, timesAz[-1]-1.0], [np.min(accelsAz), np.max(accelsAz)], ls=':', color='g', label="InPos")
    plt.xlabel("Time(sec)")
    plt.ylabel("Accel(deg/s^2)")
    plt.legend(bbox_to_anchor=(1.1, 0.8))
    plt.show()
    # plt.savefig(f"/home/craiglagegit/DATA/MTMount_25mar23/Accel_Comparison_{timestamp}.png")

